In [49]:
!pip install --upgrade ipywidgets

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 120.9 MB/s eta 0:00:00

  Attempting uninstall: widgetsnbextension

    Found existing installation: widgetsnbextension 3.6.6

    Uninstalling widgetsnbextension-3.6.6:

      Successfully uninstalled widgetsnbextension-3.6.6

  Attempting uninstall: jupyterlab_widgets

    Found existing installation: jupyterlab-widgets 1.0.0

    Uninstalling jupyterlab-widgets-1.0.0:

      Successfully uninstalled jupyterlab-widgets-1.0.0

  Attempting uninstall: ipywidgets

    Found existing installation: ipywidgets 7.8.1

    Uninstalling ipywidgets-7.8.1:

      Successfully uninstalled ipywidgets-7.8.1

   -------------------------- ------------- 2/3 [ipywidgets]
   ---------------------------------------- 3/3 [ipywidgets]




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\PC1\anaconda3\python.exe -m pip install --upgrade pip


In [53]:
!pip install notebook


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\PC1\anaconda3\python.exe -m pip install --upgrade pip


In [75]:
# Enable the widget extension for Jupyter Notebook
!jupyter nbextension enable --py widgetsnbextension --user

# Verify the extension is enabled
!jupyter nbextension list

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run script server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
  

In [77]:
# First, let's check if widgets are already available
import ipywidgets as widgets
from IPython.display import display

# Test if widgets work
button = widgets.Button(description="Test Button")
display(button)

# If the button displays, then widgets are working
# If not, let's try to enable the extension programmatically

Button(description='Test Button', style=ButtonStyle())

In [79]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from IPython.display import display

# Load data
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

# Clean title function
def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

movies["clean_title"] = movies["title"].apply(clean_title)

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf = vectorizer.fit_transform(movies["clean_title"])

# Search function
def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    return results

# Recommendation function
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

# Interactive search without widgets
def interactive_search():
    while True:
        title = input("Enter a movie title (or 'quit' to exit): ")
        if title.lower() == 'quit':
            break
        if len(title) > 3:
            try:
                results = search(title)
                if not results.empty:
                    print("\nSearch results:")
                    display(results)
                    
                    movie_id = results.iloc[0]["movieId"]
                    recommendations = find_similar_movies(movie_id)
                    
                    print("\nRecommendations based on", results.iloc[0]["title"])
                    display(recommendations)
                else:
                    print("No results found. Try a different movie title.")
            except Exception as e:
                print(f"Error: {e}")
        else:
            print("Please enter at least 4 characters.")
        print("\n" + "-"*50 + "\n")

# Run the interactive search
interactive_search()

Enter a movie title (or 'quit' to exit):  Toy story



Search results:


,movieId,title,genres,clean_title
3021,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
14813,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
59767,201588,Toy Story 4 (2019),Adventure|Animation|Children|Comedy,Toy Story 4 2019
20497,106022,Toy Story of Terror (2013),Animation|Children|Comedy,Toy Story of Terror 2013



Recommendations based on Toy Story 2 (1999)


,score,title,genres
3021,18.841924,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
2264,8.210086,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy
2669,6.868954,"Iron Giant, The (1999)",Adventure|Animation|Children|Drama|Sci-Fi
14813,6.503216,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX
3650,6.272875,Chicken Run (2000),Animation|Children|Comedy
1992,5.531892,"Little Mermaid, The (1989)",Animation|Children|Comedy|Musical|Romance
1818,5.362941,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...
2895,5.349396,Who Framed Roger Rabbit? (1988),Adventure|Animation|Children|Comedy|Crime|Fant...
0,5.287943,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3082,5.283613,Galaxy Quest (1999),Adventure|Comedy|Sci-Fi



--------------------------------------------------



Enter a movie title (or 'quit' to exit):  quit


In [81]:
movie_id = 89745

#def find_similar_movies(movie_id):
movie = movies[movies["movieId"] == movie_id]

In [83]:
ratings = pd.read_csv("ratings.csv")

In [85]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [87]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [89]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [91]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [93]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [95]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [97]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [99]:
rec_percentages

,similar,all
movieId,,
89745,1.000000,0.040459
58559,0.573393,0.148256
59315,0.530649,0.054931
79132,0.519715,0.132987
2571,0.496687,0.247010
...,...,...
47610,0.103545,0.022770
780,0.103380,0.054723
88744,0.103048,0.010383


In [101]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [103]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [105]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
17067,1.000000,0.040459,24.716368,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX,Avengers The 2012
20513,0.103711,0.005289,19.610199,106072,Thor: The Dark World (2013),Action|Adventure|Fantasy|IMAX,Thor The Dark World 2013
25058,0.241054,0.012367,19.491770,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi,Avengers Age of Ultron 2015
19678,0.216534,0.012119,17.867419,102125,Iron Man 3 (2013),Action|Sci-Fi|Thriller|IMAX,Iron Man 3 2013
16725,0.215043,0.012052,17.843074,88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War,Captain America The First Avenger 2011
16312,0.175447,0.010142,17.299824,86332,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,Thor 2011
21348,0.287608,0.016737,17.183667,110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX,Captain America The Winter Soldier 2014
25071,0.214049,0.012856,16.649399,122920,Captain America: Civil War (2016),Action|Sci-Fi|Thriller,Captain America Civil War 2016
25061,0.136017,0.008573,15.865628,122900,Ant-Man (2015),Action|Adventure|Sci-Fi,AntMan 2015
14628,0.242876,0.015517,15.651921,77561,Iron Man 2 (2010),Action|Adventure|Sci-Fi|Thriller|IMAX,Iron Man 2 2010


In [107]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [109]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from IPython.display import display

# Load data
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

# Clean title function
def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

movies["clean_title"] = movies["title"].apply(clean_title)

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf = vectorizer.fit_transform(movies["clean_title"])

# Search function
def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    return results

# Recommendation function
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

# Interactive search function (non-widget version)
def interactive_movie_search():
    print("Movie Recommendation System")
    print("Enter a movie title to get recommendations (type 'quit' to exit)")
    
    while True:
        title = input("\nEnter movie title: ")
        
        if title.lower() == 'quit':
            print("Goodbye!")
            break
            
        if len(title) > 3:  # Reduced threshold for better usability
            try:
                # Search for the movie
                results = search(title)
                
                if not results.empty:
                    print(f"\nTop search results for '{title}':")
                    for i, row in results.iterrows():
                        print(f"{i+1}. {row['title']} ({row['genres']})")
                    
                    # Get recommendations based on the first result
                    movie_id = results.iloc[0]["movieId"]
                    recommendations = find_similar_movies(movie_id)
                    
                    print(f"\nRecommendations similar to '{results.iloc[0]['title']}':")
                    display(recommendations)
                else:
                    print("No results found. Please try a different movie title.")
                    
            except Exception as e:
                print(f"An error occurred: {e}")
        else:
            print("Please enter at least 4 characters.")

# Run the interactive search
interactive_movie_search()

Movie Recommendation System
Enter a movie title to get recommendations (type 'quit' to exit)
